# Δημιουργία ενός μοντέλου παλινδρόμησης: προετοιμασία και οπτικοποίηση δεδομένων

## **Γραμμική Παλινδρόμηση για Κολοκύθες - Μάθημα 2**
#### Εισαγωγή

Τώρα που έχετε εξοπλιστεί με τα εργαλεία που χρειάζεστε για να ξεκινήσετε τη δημιουργία μοντέλων μηχανικής μάθησης με τα Tidymodels και το Tidyverse, είστε έτοιμοι να αρχίσετε να θέτετε ερωτήματα στα δεδομένα σας. Καθώς εργάζεστε με δεδομένα και εφαρμόζετε λύσεις μηχανικής μάθησης, είναι πολύ σημαντικό να κατανοήσετε πώς να θέσετε τη σωστή ερώτηση για να αξιοποιήσετε πλήρως τις δυνατότητες του συνόλου δεδομένων σας.

Σε αυτό το μάθημα, θα μάθετε:

-   Πώς να προετοιμάσετε τα δεδομένα σας για τη δημιουργία μοντέλων.

-   Πώς να χρησιμοποιήσετε το `ggplot2` για την οπτικοποίηση δεδομένων.

Η ερώτηση που θέλετε να απαντήσετε θα καθορίσει τον τύπο των αλγορίθμων μηχανικής μάθησης που θα χρησιμοποιήσετε. Και η ποιότητα της απάντησης που θα λάβετε θα εξαρτηθεί σε μεγάλο βαθμό από τη φύση των δεδομένων σας.

Ας το δούμε αυτό μέσα από μια πρακτική άσκηση.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Έργο τέχνης από την @allison_horst</figcaption>


<!--![Έργο τέχνης από την \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Έργο τέχνης από την \@allison_horst-->


## 1. Εισαγωγή δεδομένων κολοκύθας και ενεργοποίηση του Tidyverse

Θα χρειαστούμε τα παρακάτω πακέτα για να επεξεργαστούμε αυτό το μάθημα:

-   `tidyverse`: Το [tidyverse](https://www.tidyverse.org/) είναι μια [συλλογή πακέτων R](https://www.tidyverse.org/packages) σχεδιασμένη για να κάνει την επιστήμη δεδομένων πιο γρήγορη, εύκολη και διασκεδαστική!

Μπορείτε να τα εγκαταστήσετε ως εξής:

`install.packages(c("tidyverse"))`

Το παρακάτω script ελέγχει αν έχετε τα πακέτα που απαιτούνται για την ολοκλήρωση αυτής της ενότητας και τα εγκαθιστά για εσάς σε περίπτωση που λείπουν κάποια.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Τώρα, ας ενεργοποιήσουμε μερικά πακέτα και να φορτώσουμε τα [δεδομένα](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) που παρέχονται για αυτό το μάθημα!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Μια γρήγορη `glimpse()` δείχνει αμέσως ότι υπάρχουν κενά και ένας συνδυασμός από συμβολοσειρές (`chr`) και αριθμητικά δεδομένα (`dbl`). Η `Date` είναι τύπου χαρακτήρα και υπάρχει επίσης μια περίεργη στήλη που ονομάζεται `Package`, όπου τα δεδομένα είναι ένας συνδυασμός από `sacks`, `bins` και άλλες τιμές. Τα δεδομένα, στην πραγματικότητα, είναι λίγο χαοτικά 😤.

Στην πραγματικότητα, δεν είναι πολύ συνηθισμένο να λαμβάνουμε ένα σύνολο δεδομένων που είναι πλήρως έτοιμο για χρήση ώστε να δημιουργηθεί ένα μοντέλο ML απευθείας. Αλλά μην ανησυχείτε, σε αυτό το μάθημα θα μάθετε πώς να προετοιμάζετε ένα ακατέργαστο σύνολο δεδομένων χρησιμοποιώντας τις τυπικές βιβλιοθήκες της R 🧑‍🔧. Θα μάθετε επίσης διάφορες τεχνικές για να οπτικοποιείτε τα δεδομένα.📈📊
<br>

> Μια υπενθύμιση: Ο τελεστής pipe (`%>%`) εκτελεί λειτουργίες σε λογική ακολουθία περνώντας ένα αντικείμενο προς τα εμπρός σε μια συνάρτηση ή έκφραση κλήσης. Μπορείτε να σκεφτείτε τον τελεστή pipe σαν να λέει "και μετά" στον κώδικά σας.


## 2. Έλεγχος για ελλιπή δεδομένα

Ένα από τα πιο συνηθισμένα προβλήματα που αντιμετωπίζουν οι επιστήμονες δεδομένων είναι τα ελλιπή ή τα απούμενα δεδομένα. Η R αναπαριστά τις απούμενες ή άγνωστες τιμές με μια ειδική τιμή: `NA` (Not Available).

Πώς μπορούμε να γνωρίζουμε ότι το πλαίσιο δεδομένων περιέχει απούμενες τιμές;
<br>
-   Ένας απλός τρόπος θα ήταν να χρησιμοποιήσουμε τη βασική συνάρτηση της R `anyNA`, η οποία επιστρέφει λογικά αντικείμενα `TRUE` ή `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Υπέροχα, φαίνεται να λείπουν κάποια δεδομένα! Αυτό είναι ένα καλό σημείο για να ξεκινήσουμε.

-   Ένας άλλος τρόπος θα ήταν να χρησιμοποιήσετε τη συνάρτηση `is.na()`, η οποία υποδεικνύει ποια στοιχεία μιας στήλης λείπουν με μια λογική τιμή `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Εντάξει, η δουλειά έγινε, αλλά με ένα μεγάλο πλαίσιο δεδομένων όπως αυτό, θα ήταν αναποτελεσματικό και πρακτικά αδύνατο να ελεγχθούν όλες οι γραμμές και οι στήλες ξεχωριστά😴.

-   Ένας πιο διαισθητικός τρόπος θα ήταν να υπολογίσουμε το άθροισμα των ελλειπόντων τιμών για κάθε στήλη:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Πολύ καλύτερα! Υπάρχουν ελλείποντα δεδομένα, αλλά ίσως να μην έχουν σημασία για την τρέχουσα εργασία. Ας δούμε τι θα φέρει η περαιτέρω ανάλυση.

> Μαζί με τα εξαιρετικά σύνολα πακέτων και συναρτήσεων, η R διαθέτει πολύ καλή τεκμηρίωση. Για παράδειγμα, χρησιμοποιήστε `help(colSums)` ή `?colSums` για να μάθετε περισσότερα σχετικά με τη συνάρτηση.


## 3. Dplyr: Μια Γραμματική για τον Χειρισμό Δεδομένων

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Έργο τέχνης από την @allison_horst</figcaption>


<!--![Έργο τέχνης από την \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Έργο τέχνης από την \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), ένα πακέτο του Tidyverse, είναι μια γραμματική για τη διαχείριση δεδομένων που παρέχει ένα συνεπές σύνολο ρημάτων για να σας βοηθήσει να λύσετε τις πιο συνηθισμένες προκλήσεις διαχείρισης δεδομένων. Σε αυτή την ενότητα, θα εξερευνήσουμε μερικά από τα ρήματα του dplyr!  
<br>


#### dplyr::select()

Η `select()` είναι μια συνάρτηση στο πακέτο `dplyr` που σας βοηθά να επιλέξετε στήλες για να διατηρήσετε ή να αποκλείσετε.

Για να κάνετε το πλαίσιο δεδομένων σας πιο εύχρηστο, αφαιρέστε αρκετές από τις στήλες του χρησιμοποιώντας τη `select()`, διατηρώντας μόνο τις στήλες που χρειάζεστε.

Για παράδειγμα, σε αυτή την άσκηση, η ανάλυσή μας θα περιλαμβάνει τις στήλες `Package`, `Low Price`, `High Price` και `Date`. Ας επιλέξουμε αυτές τις στήλες.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

Το `mutate()` είναι μια συνάρτηση του πακέτου `dplyr`, η οποία σας βοηθά να δημιουργήσετε ή να τροποποιήσετε στήλες, διατηρώντας παράλληλα τις υπάρχουσες στήλες.

Η γενική δομή του `mutate` είναι:

`data %>%   mutate(new_column_name = what_it_contains)`

Ας δοκιμάσουμε το `mutate` χρησιμοποιώντας τη στήλη `Date` και εκτελώντας τις παρακάτω ενέργειες:

1.  Μετατροπή των ημερομηνιών (που είναι αυτή τη στιγμή τύπου χαρακτήρα) σε μορφή μήνα (οι ημερομηνίες είναι σε μορφή ΗΠΑ, δηλαδή `MM/DD/YYYY`).

2.  Εξαγωγή του μήνα από τις ημερομηνίες σε μια νέα στήλη.

Στη γλώσσα R, το πακέτο [lubridate](https://lubridate.tidyverse.org/) διευκολύνει την εργασία με δεδομένα τύπου Ημερομηνία-Ώρα. Έτσι, ας χρησιμοποιήσουμε τις `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` για να πετύχουμε τους παραπάνω στόχους. Μπορούμε να αφαιρέσουμε τη στήλη `Date`, καθώς δεν θα τη χρειαστούμε ξανά στις επόμενες ενέργειες.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Ουάου! 🤩

Στη συνέχεια, ας δημιουργήσουμε μια νέα στήλη `Price`, η οποία αντιπροσωπεύει τη μέση τιμή μιας κολοκύθας. Τώρα, ας πάρουμε τον μέσο όρο των στηλών `Low Price` και `High Price` για να γεμίσουμε τη νέα στήλη Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Ναι!💪

"Αλλά περίμενε!", θα πεις αφού ρίξεις μια ματιά σε όλο το σύνολο δεδομένων με `View(pumpkins)`, "Κάτι φαίνεται περίεργο εδώ!"🤔

Αν κοιτάξεις τη στήλη `Package`, οι κολοκύθες πωλούνται σε πολλές διαφορετικές διαμορφώσεις. Κάποιες πωλούνται σε μέτρα `1 1/9 bushel`, κάποιες σε μέτρα `1/2 bushel`, κάποιες ανά κολοκύθα, κάποιες ανά λίβρα, και κάποιες σε μεγάλες κουτιά με διαφορετικά πλάτη.

Ας το επαληθεύσουμε αυτό:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Καταπληκτικό!👏

Τα κολοκύθια φαίνεται να είναι πολύ δύσκολο να ζυγιστούν με συνέπεια, οπότε ας τα φιλτράρουμε επιλέγοντας μόνο τα κολοκύθια που περιέχουν τη λέξη *bushel* στη στήλη `Package` και ας τα τοποθετήσουμε σε ένα νέο πλαίσιο δεδομένων `new_pumpkins`.


#### dplyr::filter() και stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): δημιουργεί ένα υποσύνολο των δεδομένων που περιέχει μόνο τις **γραμμές** που ικανοποιούν τις συνθήκες σας, σε αυτή την περίπτωση, κολοκύθες με τη λέξη *bushel* στη στήλη `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): ανιχνεύει την παρουσία ή την απουσία ενός μοτίβου μέσα σε μια συμβολοσειρά.

Το πακέτο [`stringr`](https://github.com/tidyverse/stringr) παρέχει απλές συναρτήσεις για κοινές λειτουργίες με συμβολοσειρές.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Μπορείτε να δείτε ότι έχουμε περιορίσει τα δεδομένα σε περίπου 415 σειρές που περιέχουν κολοκύθες ανά καλάθι.🤩  
<br>


#### dplyr::case_when()

**Αλλά περίμενε! Υπάρχει κάτι ακόμα που πρέπει να κάνεις**

Παρατήρησες ότι η ποσότητα ανά bushel διαφέρει ανά γραμμή; Πρέπει να κανονικοποιήσεις την τιμολόγηση ώστε να δείχνεις την τιμή ανά bushel, όχι ανά 1 1/9 ή 1/2 bushel. Ώρα να κάνεις λίγα μαθηματικά για να την τυποποιήσεις.

Θα χρησιμοποιήσουμε τη συνάρτηση [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) για να *μετασχηματίσουμε* τη στήλη Price ανάλογα με ορισμένες συνθήκες. Το `case_when` σου επιτρέπει να διαχειριστείς πολλαπλές δηλώσεις `if_else()` με τρόπο που να είναι πιο ευέλικτος.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Τώρα, μπορούμε να αναλύσουμε την τιμή ανά μονάδα με βάση τη μέτρηση σε bushel. Όλη αυτή η μελέτη των bushel κολοκύθας, ωστόσο, δείχνει πόσο `σημαντικό` είναι να `κατανοήσετε τη φύση των δεδομένων σας`!

> ✅ Σύμφωνα με [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), το βάρος ενός bushel εξαρτάται από τον τύπο του προϊόντος, καθώς είναι μέτρηση όγκου. "Ένα bushel ντομάτας, για παράδειγμα, υποτίθεται ότι ζυγίζει 56 λίβρες... Φύλλα και χόρτα καταλαμβάνουν περισσότερο χώρο με λιγότερο βάρος, οπότε ένα bushel σπανάκι ζυγίζει μόνο 20 λίβρες." Είναι αρκετά περίπλοκο! Ας μην ασχοληθούμε με τη μετατροπή από bushel σε λίβρες και ας καθορίσουμε την τιμή ανά bushel. Όλη αυτή η μελέτη των bushel κολοκύθας, ωστόσο, δείχνει πόσο σημαντικό είναι να κατανοήσετε τη φύση των δεδομένων σας!
>
> ✅ Παρατηρήσατε ότι οι κολοκύθες που πωλούνται ανά μισό bushel είναι πολύ ακριβές; Μπορείτε να καταλάβετε γιατί; Υπόδειξη: οι μικρές κολοκύθες είναι πολύ πιο ακριβές από τις μεγάλες, πιθανότατα επειδή υπάρχουν πολύ περισσότερες ανά bushel, δεδομένου του ανεκμετάλλευτου χώρου που καταλαμβάνει μία μεγάλη, κούφια κολοκύθα για πίτα.


Τώρα, τέλος, για χάρη της περιπέτειας 💁‍♀️, ας μετακινήσουμε τη στήλη Month στην πρώτη θέση, δηλαδή `πριν` από τη στήλη `Package`.

Η `dplyr::relocate()` χρησιμοποιείται για να αλλάξουμε τις θέσεις των στηλών.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Μπράβο!👌 Τώρα έχεις ένα καθαρό, τακτοποιημένο σύνολο δεδομένων πάνω στο οποίο μπορείς να χτίσεις το νέο σου μοντέλο παλινδρόμησης!  
<br>


## 4. Οπτικοποίηση δεδομένων με ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Γραφικό από τον Dasani Madipalli</figcaption>


<!--![Γραφικό από τον Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Υπάρχει μια *σοφή* ρήση που λέει:

> "Το απλό γράφημα έχει φέρει περισσότερες πληροφορίες στο μυαλό του αναλυτή δεδομένων από οποιαδήποτε άλλη μέθοδο." --- John Tukey

Μέρος του ρόλου του επιστήμονα δεδομένων είναι να δείξει την ποιότητα και τη φύση των δεδομένων με τα οποία εργάζεται. Για να το πετύχει αυτό, συχνά δημιουργεί ενδιαφέρουσες οπτικοποιήσεις, ή διαγράμματα, γραφήματα και πίνακες, που παρουσιάζουν διαφορετικές πτυχές των δεδομένων. Με αυτόν τον τρόπο, μπορεί να δείξει οπτικά σχέσεις και κενά που διαφορετικά είναι δύσκολο να εντοπιστούν.

Οι οπτικοποιήσεις μπορούν επίσης να βοηθήσουν στον καθορισμό της τεχνικής μηχανικής μάθησης που είναι πιο κατάλληλη για τα δεδομένα. Ένα διάγραμμα διασποράς που φαίνεται να ακολουθεί μια γραμμή, για παράδειγμα, υποδεικνύει ότι τα δεδομένα είναι καλός υποψήφιος για μια άσκηση γραμμικής παλινδρόμησης.

Η R προσφέρει αρκετά συστήματα για τη δημιουργία γραφημάτων, αλλά το [`ggplot2`](https://ggplot2.tidyverse.org/index.html) είναι ένα από τα πιο κομψά και ευέλικτα. Το `ggplot2` σας επιτρέπει να συνθέσετε γραφήματα **συνδυάζοντας ανεξάρτητα στοιχεία**.

Ας ξεκινήσουμε με ένα απλό διάγραμμα διασποράς για τις στήλες Price και Month.

Σε αυτή την περίπτωση, θα ξεκινήσουμε με το [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), θα παρέχουμε ένα σύνολο δεδομένων και μια αισθητική αντιστοίχιση (με το [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) και στη συνέχεια θα προσθέσουμε επίπεδα (όπως το [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) για διαγράμματα διασποράς.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Είναι αυτή μια χρήσιμη γραφική παράσταση 🤷; Σας εκπλήσσει κάτι σε αυτήν;

Δεν είναι ιδιαίτερα χρήσιμη, καθώς το μόνο που κάνει είναι να εμφανίζει τα δεδομένα σας ως μια διασπορά σημείων σε έναν συγκεκριμένο μήνα.
<br>


### **Πώς το κάνουμε χρήσιμο;**

Για να εμφανίζονται χρήσιμα δεδομένα στα γραφήματα, συνήθως χρειάζεται να ομαδοποιήσετε τα δεδομένα με κάποιον τρόπο. Για παράδειγμα, στην περίπτωσή μας, η εύρεση της μέσης τιμής των κολοκυθών για κάθε μήνα θα παρείχε περισσότερες πληροφορίες για τα υποκείμενα μοτίβα στα δεδομένα μας. Αυτό μας οδηγεί σε μία ακόμη σύντομη αναφορά στη **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Η ομαδοποιημένη συναθροιστική επεξεργασία στην R μπορεί να υπολογιστεί εύκολα χρησιμοποιώντας

`dplyr::group_by() %>% summarize()`

-   Το `dplyr::group_by()` αλλάζει τη μονάδα ανάλυσης από ολόκληρο το σύνολο δεδομένων σε μεμονωμένες ομάδες, όπως ανά μήνα.

-   Το `dplyr::summarize()` δημιουργεί ένα νέο πλαίσιο δεδομένων με μία στήλη για κάθε μεταβλητή ομαδοποίησης και μία στήλη για κάθε στατιστικό σύνοψης που έχετε καθορίσει.

Για παράδειγμα, μπορούμε να χρησιμοποιήσουμε το `dplyr::group_by() %>% summarize()` για να ομαδοποιήσουμε τις κολοκύθες σε ομάδες βάσει της στήλης **Month** και στη συνέχεια να βρούμε τη **μέση τιμή** για κάθε μήνα.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Συνοπτικά!✨

Κατηγοριοποιημένα χαρακτηριστικά, όπως οι μήνες, αποδίδονται καλύτερα χρησιμοποιώντας ένα γράφημα με μπάρες 📊. Τα layers που είναι υπεύθυνα για τα γραφήματα με μπάρες είναι `geom_bar()` και `geom_col()`. Συμβουλευτείτε το `?geom_bar` για να μάθετε περισσότερα.

Ας φτιάξουμε ένα!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Αυτή είναι μια πιο χρήσιμη απεικόνιση δεδομένων! Φαίνεται να δείχνει ότι η υψηλότερη τιμή για τις κολοκύθες εμφανίζεται τον Σεπτέμβριο και τον Οκτώβριο. Συμφωνεί αυτό με τις προσδοκίες σας; Γιατί ή γιατί όχι;

Συγχαρητήρια για την ολοκλήρωση του δεύτερου μαθήματος 👏! Προετοιμάσατε τα δεδομένα σας για τη δημιουργία μοντέλου και στη συνέχεια ανακαλύψατε περισσότερες πληροφορίες χρησιμοποιώντας απεικονίσεις!



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
